Working on adding a lot of features just to see if it can get the score up regardless of how complicated or where the data is coming from

Features that I will be adding

* Taget given surface counts
* words from the target and source document
  * possible back prop into these vectors, idea is to replace tf-idf with some nn and back prop here
* using a linear layer near the output to combine the features

* adding the indicator features from the berkeley entity system
  * I guess that these will be on the final linear layer
  
* the tesor product type stuff did not really work, going to try and maybe increase the dim of the convs somehow

* going to add the join system since the features on the queries appear to matter a lot
    * The score of the query is multiplied by the score of the join

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from theano import *
from lasagne.layers import InputLayer, get_output
import lasagne
import lasagne.layers
import theano.tensor as T
import theano
import numpy as np
from helpers import SimpleMaxingLayer, SimpleAverageLayer
from wordvecs import WordVectors, EmbeddingLayer, WordTokenizer
import json
import re
import random

theano.config.floatX = 'float32'
#theano.config.linker = 'cvm_nogc'
theano.config.openmp = True
theano.config.openmp_elemwise_minsize = 20000


# for debugging
# theano.config.exception_verbosity = 'high'
# theano.config.optimizer = 'None'

Couldn't import dot_parser, loading of dot files will not be possible.


In [3]:
with open('/data/matthew/external-wiki-wplinks6.json') as f:
    externalWikiJ = json.load(f)
    queries = externalWikiJ['queries']
    featuresNames = externalWikiJ['featIndex']

In [21]:
def evalNumPossible(qus):
    total = 0
    possible = 0
    for qu in qus.values():
        for q in qu.values():
            total += 1
            if len(set(q['gold']) & set(q['vals'].keys())) != 0:
                possible += 1
    return float(possible) / total

evalNumPossible(queries)

0.9933534743202417

In [5]:
len(featuresNames)

9954

In [6]:
len(queries)

463

In [7]:
sum([any([g['gold'] for g in v.values()]) for v in queries.values()])

463

In [8]:
sum(sum(not g['training'] for g in v.values()) for v in queries.values())

485

In [9]:
sum(sum(g['training'] for g in v.values()) for v in queries.values())

4480

In [10]:
len(featuresNames)

9954

In [11]:
max(max(max(max(a[1]) for a in g['vals'].values()) for g in v.values()) for v in queries.values())

[67]

In [12]:
# for q,v in queries.items():
#     if not v.values()[0]['training']:
#         del queries[q]

In [13]:
# # load the samples from the provided testing set

# with open('/data/matthew/external-wiki-testing.json') as f:
#     queries.update(**json.load(f)['queries'])

In [14]:
for q,v in queries.items():
    for v2 in v.values():
        gi = set(g.replace('_', ' ') for g in v2['gold']) | set(v2['gold'])
        v2['gold'] = list(gi)

In [15]:
wordvectors = WordVectors(
    fname="/data/matthew/enwiki-20141208-pages-articles-multistream-links7-output1.bin",
    redir_fname='/data/matthew/enwiki-20141208-pages-articles-multistream-redirect7.json',
    negvectors=False,
    sentence_length=200,
)
wordvectors.add_unknown_words = False

In [16]:
# with open('/data/matthew/enwiki-20141208-pages-articles-multistream-redirects5.json') as f:
#     page_redirects = json.load(f)
page_redirects = wordvectors.redirects

In [17]:
len(wordvectors.vectors)

4056055

In [18]:
with open('/data/matthew/enwiki-20141208-pages-articles-multistream-surface-counts7.json') as f:
    surface_counts = json.load(f)

In [19]:
# try and make the surfaces items match what we are looking for
surface_counts_re = re.compile('([\.,!\?])')
for sk in surface_counts.keys():
    nsk = sk.replace('(', '-lrb-').replace(')', '-rrb-')
    nsk = surface_counts_re.sub(' \\1', nsk)
    if nsk != sk:
        surface_counts[nsk] = surface_counts[sk]

In [20]:
# clean up whitespace around targets
def cleanWhitespaces():
    for qu in queries.values():
        for en in qu.values():
            en['gold'] = [g.strip() for g in en['gold']]
            nv = {}
            for k, v in en['vals'].iteritems():
                nv[k.strip()] = v
            en['vals'] = nv
cleanWhitespaces()

In [22]:
from wikireader import WikiRegexes, WikipediaReader

In [23]:
def PreProcessedQueries(wikipedia_dump_fname, vectors=wordvectors, queries=queries, redirects=page_redirects, surface=surface_counts):
    
    get_words = re.compile('[^a-zA-Z0-9 ]')
    get_link = re.compile('.*?\[(.*?)\].*?')
    
    wordvec = WordTokenizer(vectors, sentence_length=200)
    documentvec = WordTokenizer(vectors, sentence_length=1)
    
    queried_pages = set()
    for docs, q in queries.iteritems():
        wordvec.tokenize(docs)
        for sur, v in q.iteritems():
            wrds_sur = get_words.sub(' ', sur)
            wordvec.tokenize(wrds_sur)
            link_sur = get_link.match(sur).group(1)
            wordvec.tokenize(link_sur)
            for link in v['vals'].keys():
                wrds = get_words.sub(' ', link)
                wordvec.tokenize(wrds)
                tt = WikiRegexes.convertToTitle(link)
                documentvec.get_location(tt)
                queried_pages.add(tt)

    added_pages = set()
    for title in queried_pages:
        if title in redirects:
            #wordvec.tokenize(self.redirects[title])
            documentvec.get_location(redirects[title])
            added_pages.add(redirects[title])
    queried_pages |= added_pages
    
    for w in queried_pages:
        wordvec.tokenize(get_words.sub(' ', w))

    page_content = {}

    class GetWikipediaWords(WikipediaReader, WikiRegexes):

        def readPage(ss, title, content, namespace):
            if namespace != 0:
                return
            tt = ss.convertToTitle(title)
            if tt in queried_pages:
                cnt = ss._wikiToText(content)
                page_content[tt] = wordvec.tokenize(cnt)

    GetWikipediaWords(wikipedia_dump_fname).read()
    
    rr = redirects
    rq = queried_pages
    rc = page_content
    rs = surface

    class PreProcessedQueriesCls(object):
        
        wordvecs = wordvec
        documentvecs = documentvec
        queries = queries
        redirects = rr
        queried_pages = rq
        page_content = rc
        surface_counts = rs
        
        
    return PreProcessedQueriesCls

In [24]:
%time basePreProcessedQueries = PreProcessedQueries('/data/matthew/enwiki-20141208-pages-articles-multistream.xml')

CPU times: user 6min 8s, sys: 14.9 s, total: 6min 23s
Wall time: 6min 30s


In [25]:
len(wordvectors.vectors)

4056055

In [26]:
len(basePreProcessedQueries.wordvecs.reverse_word_location)

146885

In [27]:
for qu in queries.values():
    for en in qu.values():
        en['boosted'] = 1.0

In [28]:
def cleanUpMultipleLinks():
    for qu in queries.values():
        for en in qu.values():
            gold_pages = en['gold']
            gold_titles = [WikiRegexes.convertToTitle(g.strip()) for g in gold_pages]
            gold_titles = [page_redirects.get(g, g) for g in gold_titles]
            pages = set()
            for p in en['vals'].keys():
                wiki_title = WikiRegexes.convertToTitle(p)
                wiki_title = page_redirects.get(wiki_title, wiki_title)
                if wiki_title in gold_titles and p not in en['gold']:
                    del en['vals'][p]
                elif wiki_title not in pages:
                    pages.add(wiki_title)
                else:
                    del en['vals'][p]
cleanUpMultipleLinks()

In [29]:
sum(len(q) for q in queries)

314614

In [30]:
# def removeSingleLinkTargets():
#     "items that the surface link set only has a single item are trival, so remove"
#     get_link = re.compile('.*?\[(.*?)\].*?')
#     for quk, qu in queries.items():
#         for sur in qu.keys():
#             surlink = get_link.match(sur).group(1)
#             surmatch = surlink.lower()
#             surcounts = surface_counts.get(surmatch)
#             if surcounts and len(surcounts) == 1:
#                 # this is trival since there is only one item
#                 del qu[sur]
#         if not qu:
#             # we removed all the links on this page, remove it otherwise the program crashes
#             del queries[quk]
# removeSingleLinkTargets()

In [54]:
class EntityVectorLinkExp(basePreProcessedQueries):

    batch_size = 250 #20000
    num_training_items = 500000 #200000
    dim_compared_vec = 150  # 100

    def __init__(self):
        self.sentence_length = self.wordvecs.sentence_length
        self.sentence_length_short = 10
        self.document_length = 100
        
        self.num_words_to_use_conv = 5
        self.enable_boosting = False
        self.num_negative_target_samples = 0 #1
        #self.enable_match_surface = False
        #self.enable_link_counts = True
        self.enable_train_wordvecs = False
        self.enable_cap_boosting = True
        
        self.num_indicator_features = len(featuresNames)
        
        self.main_nl = lasagne.nonlinearities.softmax# leaky_rectify
        
        self.impossible_query = featuresNames.index('Impossible')

        self._setup()

    def _setup(self):
        
        self.all_params = []
        
        self.x_document_input = T.imatrix('x_doc')  # words from the source document

        self.x_document_id = T.ivector('x_doc_id')  # index of which source doucment this is from
        self.x_surface_text_input = T.imatrix('x_surface_link')  # text of the surface link
        self.x_surface_context_input = T.imatrix('x_surface_cxt')  #  words surrounding the surface link

        self.x_target_input = T.ivector('x_target')  # id of the target vector
        self.x_target_words = T.imatrix('x_target_words')  # words from the target title link
        self.x_matches_surface = T.ivector('x_match_surface')  # indicator if the target title matches the surface
        self.x_matches_counts = T.imatrix('x_matches_counts')  # info about the link counts
        self.x_target_document_words = T.imatrix('x_target_document_words')  # words from the body of target document
        self.x_link_id = T.ivector('x_link_id')  # indx of what link to compare to in the matrix

        #self.x_indicator_features = T.matrix('x_indicator_features', dtype='int8')
        
        self.x_denotaiton_features = T.matrix('x_denotation_ind_feats', dtype='int8')  # the joint denotation query features
        self.x_query_featurs = T.matrix('x_query_ind_feats', dtype='int8')  # the query features
        self.x_query_link_id = T.ivector('x_match_query')  # the query that a denotation links to
        self.x_denotation_ranges = T.imatrix('x_denotation_ranges')  # the range of joint denotations to sum over
        
        self.x_target_link_id = T.ivector('x_match_target')  # the target document that maches with a given denotation
        
        #self.y_score = T.vector('y')
        self.y_answer = T.ivector('y_ans')  # (Not used) contains the location of the gold answer so we can compute the loss
        self.y_grouping = T.imatrix('y_grouping')  # matrix containing [start_idx, end_idx, gold_idx]
        self.y_boosted = T.vector('y_boosted')  # only used if boosting enabled, vector of how much to boost items

        self.embedding_W = theano.shared(self.wordvecs.get_numpy_matrix().astype(theano.config.floatX),name='embedding_W')
        self.embedding_W_docs = theano.shared(self.documentvecs.get_numpy_matrix().astype(theano.config.floatX),name='embedding_W_docs')
        
#         def convSoftMax(t):
#             shape = t.shape  # (document_sample, num_filters, output_rows, output_cols)
#             new_shape = (shape[0] * shape[2] * shape[3], shape[1])
#             return T.nnet.softmax(t.reshape(new_shape)).reshape(shape)
        
#             return lasagne.nonlinearities.leaky_rectify(t)

        def augRectify(x):
            # if x is zero, then the gradient failes due to computation: x / |x|
            #return lasagne.nonlinearities.leaky_rectify(x + .001) #- .001
            return T.maximum(x, -.01 * x)
        
        simpleConvNonLin = augRectify # lasagne.nonlinearities.rectify# leaky_rectify
        
        self.document_l = lasagne.layers.InputLayer(
            (None,self.document_length),
            input_var=self.x_document_input
        )

        self.document_embedding_l = EmbeddingLayer(
            self.document_l,
            W=self.embedding_W,
            add_word_params=self.enable_train_wordvecs,
        )
        
        self.document_simple_conv1_l = lasagne.layers.Conv2DLayer(
            self.document_embedding_l,
            num_filters=self.dim_compared_vec,
            filter_size=(2, self.wordvecs.vector_size),
            name='document_simple_conv',
            nonlinearity=simpleConvNonLin,#lasagne.nonlinearities.leaky_rectify,
        )
        
        self.document_simple_sum_l = lasagne.layers.Pool2DLayer(
            #lasagne.layers.reshape(self.document_embedding_l, ([0],[3],[2],1)),
            self.document_simple_conv1_l,
            name='document_simple_pool',
            pool_size=(self.document_length - 2, 1),
            mode='sum',
        )
        
#         self.document_conv1_l = lasagne.layers.Conv2DLayer(
#             self.document_embedding_l,
#             num_filters=30,  # was 75, 100, 500
#             filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
#             name='document_conv1',
#             nonlinearity=self.main_nl,# lasagne.nonlinearities.softmax,  # was leaky_rectify
#         )

#         self.document_max_l = lasagne.layers.Pool2DLayer(
#             self.document_conv1_l,
#             name='document_pool1',
#             pool_size=(self.document_length - self.num_words_to_use_conv, 1),
#             mode='max',  # was sum
#         )

#         document_output_length = 25 # was 100, 200
        
#         self.document_dens1 = lasagne.layers.DenseLayer(
#             self.document_max_l,
#             num_units=document_output_length,
#             name='doucment_dens1',
#             nonlinearity=lasagne.nonlinearities.leaky_rectify,
#         )

        self.document_output = lasagne.layers.get_output(
            lasagne.layers.reshape(self.document_simple_sum_l, ([0],-1)))
        
        self.all_params += lasagne.layers.get_all_params(self.document_simple_sum_l)
    
    
        ##########################################
        ## surface text

        self.surface_context_l = lasagne.layers.InputLayer(
            (None, self.sentence_length),
            input_var=self.x_surface_context_input,
        )

        self.surface_context_embedding_l = EmbeddingLayer(
            self.surface_context_l,
            W=self.embedding_W,
            add_word_params=self.enable_train_wordvecs,
        )

        self.surface_context_conv1_l = lasagne.layers.Conv2DLayer(
            self.surface_context_embedding_l,
            num_filters=self.dim_compared_vec,  # was 300
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            name='surface_cxt_conv1',
            nonlinearity=simpleConvNonLin,# self.main_nl,
        )

        self.surface_context_pool1_l = lasagne.layers.Pool2DLayer(
            self.surface_context_conv1_l,
            name='surface_cxt_pool1',
            pool_size=(self.sentence_length - self.num_words_to_use_conv, 1),
            mode='max', #sum',
        )
    
        self.surface_output = lasagne.layers.get_output(
            lasagne.layers.reshape(self.surface_context_pool1_l, ([0], -1))
        )
        
        self.all_params += lasagne.layers.get_all_params(self.surface_context_pool1_l)

        self.surface_input_l = lasagne.layers.InputLayer(
            (None, self.sentence_length_short),
            input_var=self.x_surface_text_input
        )

        self.surface_embedding_l = EmbeddingLayer(
            self.surface_input_l,
            W=self.embedding_W,
            add_word_params=self.enable_train_wordvecs,
        )

        self.surface_conv1_l = lasagne.layers.Conv2DLayer(
            self.surface_embedding_l,
            num_filters=self.dim_compared_vec,  # was 300
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            name='surface_conv1',
            nonlinearity=simpleConvNonLin, #self.main_nl,
        )

        self.surface_pool1_l = lasagne.layers.Pool2DLayer(
            self.surface_conv1_l,
            name='surface_pool1',
            pool_size=(self.sentence_length_short - self.num_words_to_use_conv, 1),
            mode='sum',
        )
    
        self.surface_words_output = lasagne.layers.get_output(
            lasagne.layers.reshape(self.surface_pool1_l, ([0], -1))
        )
        
        self.all_params += lasagne.layers.get_all_params(self.surface_pool1_l)

        ##################################################
        ## merge the documents with the surface info

        
        ###################################################
        ## dealing with the target side

        matched_surface_reshaped = self.x_matches_surface.reshape(
            (self.x_matches_surface.shape[0], 1, 1, 1)).astype(theano.config.floatX)

        self.target_input_l = lasagne.layers.InputLayer(
            (None,),
            input_var=self.x_target_input
        )
        
        #################################
        ## target indicators features

        self.target_matched_surface_input_l = lasagne.layers.InputLayer(
            (None,1,1,1),
            input_var=matched_surface_reshaped,
        )
        
        self.target_matched_counts_input_l = lasagne.layers.InputLayer(
            (None,5),
            input_var=self.x_matches_counts.astype(theano.config.floatX),
        )

#         # embedding of the target documents
#         self.target_embedding_l = EmbeddingLayer(
#             lasagne.layers.reshape(self.target_input_l, ([0], 1)),
#             W=self.embedding_W_docs,
#             add_word_params=False,
#         )
        
#         self.target_embedding_dens_l = lasagne.layers.DenseLayer(
#             lasagne.layers.reshape(self.target_embedding_l, ([0], -1)),
#             name='target_embedding_dens',
#             num_units=self.dim_compared_vec,
#             nonlinearity=augRectify,# lasagne.nonlinearities.leaky_rectify,  # should be compariable to the simpleConvNonLin
#         )
        
#         self.target_embedding_out = lasagne.layers.get_output(
#             lasagne.layers.reshape(self.target_embedding_dens_l, ([0],-1)),
#         )

        # words from the title of the target
        self.target_words_input_l = lasagne.layers.InputLayer(
            (None,self.sentence_length_short),
            input_var=self.x_target_words,
        )

        self.target_words_embedding_l = EmbeddingLayer(
            self.target_words_input_l,
            W=self.embedding_W,
            add_word_params=self.enable_train_wordvecs,
        )

        self.target_words_conv1_l = lasagne.layers.Conv2DLayer(
            self.target_words_embedding_l,
            name='target_wrds_conv1',
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            num_filters=self.dim_compared_vec,  # was 75, 150, 350
            nonlinearity=simpleConvNonLin,# self.main_nl,# lasagne.nonlinearities.leaky_rectify,
        )

        self.target_words_pool1_l = lasagne.layers.Pool2DLayer(
            self.target_words_conv1_l,
            name='target_wrds_pool1',
            pool_size=(self.sentence_length_short - self.num_words_to_use_conv, 1),
            mode='sum',  # was sum
        )
        
        self.target_title_out = lasagne.layers.get_output(
            lasagne.layers.reshape(self.target_words_pool1_l, ([0],-1))
        )
        
        self.all_params += lasagne.layers.get_all_params(self.target_words_pool1_l)
        
        
        # words from the body of the target
        self.target_body_words_input_l = lasagne.layers.InputLayer(
            (None,self.sentence_length),
            input_var=self.x_target_document_words,
        )
        
        self.target_body_words_embedding_l = EmbeddingLayer(
            self.target_body_words_input_l,
            W=self.embedding_W,
            add_word_params=self.enable_train_wordvecs,
        )
        
        self.target_body_simple_conv1_l = lasagne.layers.Conv2DLayer(
            self.target_body_words_embedding_l,
            name='target_body_simple_conv',
            filter_size=(2, self.wordvecs.vector_size),
            num_filters=self.dim_compared_vec,
            nonlinearity=simpleConvNonLin,# self.main_nl,# lasagne.nonlinearities.leaky_rectify,
        )
        
        self.target_body_simple_sum_l = lasagne.layers.Pool2DLayer(
            #lasagne.layers.reshape(self.target_body_words_embedding_l, ([0],[3],[2],1)),
            self.target_body_simple_conv1_l,
            name='target_body_simple_sum',
            pool_size=(self.sentence_length - 2, 1),
            mode='sum',
        )
        
#         self.target_body_words_conv1_l = lasagne.layers.Conv2DLayer(
#             self.target_body_words_embedding_l,
#             name='target_body_wrds_conv1',
#             filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
#             num_filters=150,
#             nonlinearity=lasagne.nonlinearities.leaky_rectify,
#         )
        
#         self.target_body_words_pool1_l = lasagne.layers.Pool2DLayer(
#             self.target_body_words_conv1_l,
#             name='target_body_wrds_pool1',
#             pool_size=(self.sentence_length - self.num_words_to_use_conv, 1),
#             mode='max',
#         )
    
#         self.target_merge_l = lasagne.layers.ConcatLayer(
#             [lasagne.layers.reshape(self.target_words_pool1_l, ([0], [1])),
#              lasagne.layers.reshape(self.target_body_words_pool1_l, ([0], [1])),
#             # lasagne.layers.reshape(self.target_embedding_l, ([0], [3]))
#             ]
#         )

        self.target_out = lasagne.layers.get_output(
            lasagne.layers.reshape(self.target_body_simple_sum_l, ([0],-1)))
        
        self.all_params += lasagne.layers.get_all_params(self.target_body_simple_sum_l)

        #########################################################
        ## compute the cosine distance between the two layers
       
        # source body
        self.source_aligned_l = self.document_output[self.x_document_id,:][self.x_link_id,:] #self.source_out[self.x_link_id, :]
        # source context
        self.source_context_aligned_l = self.surface_output[self.x_link_id,:]
        # source surface words
        self.source_surface_words_aligned_l = self.surface_words_output[self.x_link_id,:]
        
        # this uses scan internally, which means that it comes back into python code to run the loop.....fml
#         self.dotted_vectors =  T.batched_dot(self.target_out, self.source_aligned_l)
        # diag also does not support a C version.........
        #self.dotted_vectors = T.dot(self.target_out, self.source_aligned_l.T).diagonal()

        def augNorm(v):
            return T.basic.pow(T.basic.pow(T.basic.abs_(v), 2).sum(axis=1) + .001, .5)

#         self.res_l = self.dotted_vectors / (augNorm(self.target_out) * augNorm(self.source_aligned_l) + .001)
        
#         self.res_cap = T.clip((T.tanh(self.res_l) + 1) / 2, .001, .999)
        
        def cosinsim(a, b):
            dotted = T.batched_dot(a, b)
            return dotted / (augNorm(a) * augNorm(b))
        
        ##############################################
        ## tensor product stuff
        
#         def tensorP(a,b):
#             res, _ = theano.scan(
#                 fn=lambda x_vec, y_vec, x_norm, y_norm: T.concatenate(
#                     [x_vec, y_vec, 
#                      T.outer(x_vec / x_norm, y_vec / y_norm).flatten()]
#                 ),
#                 outputs_info=None,
#                 sequences=[a,b, augNorm(a), augNorm(b)],
#                 non_sequences=None
#             )
#             return res
        
                
        def comparedVLayers(a, b):
            dv = cosinsim(a, b)
            #dv = (a[:,0] + .001) * (b[:,0] + .001)
            return lasagne.layers.InputLayer(
                (None,1),
                input_var=dv.reshape((dv.shape[0], 1))
            )
        
#         def comparedVLayers2(a,b):
#             dv = theano.gradient.disconnected_grad(tensorP(a,b))  # just see how well this performs without learning this
#             return lasagne.layers.InputLayer(
#                 (None,self.dim_compared_vec ** 2 + self.dim_compared_vec*2 ),
#                 input_var=dv
#             )
        
        
        
    
        ######################################################
        ## indicator feature input
    
#         self.indicator_feat_l = lasagne.layers.InputLayer(
#             (None, self.num_indicator_features),
#             input_var=self.x_indicator_features.astype(theano.config.floatX),
#         )

        self.cosine_combined = lasagne.layers.concat(
            [
               comparedVLayers(self.target_out, self.source_aligned_l),
               comparedVLayers(self.target_out, self.source_context_aligned_l),
               comparedVLayers(self.target_out, self.source_surface_words_aligned_l),
                
               comparedVLayers(self.target_title_out, self.source_aligned_l),
               comparedVLayers(self.target_title_out, self.source_context_aligned_l),
               comparedVLayers(self.target_title_out, self.source_surface_words_aligned_l),
            ],
            axis=1
        )
    
        self.cosine_weighted = lasagne.layers.DenseLayer(
            self.cosine_combined,
            name='cosine_dens1',
            num_units=1,
            b=None,
            nonlinearity=lasagne.nonlinearities.linear,
        )
        
#         self.cosine_weighted.W.get_value(borrow=True)[:] += 1
        
        self.cosine_output = lasagne.layers.get_output(
            lasagne.layers.reshape(self.cosine_weighted, (-1,)))
        
        self.all_params += lasagne.layers.get_all_params(self.cosine_weighted)

        self.query_feat_l = lasagne.layers.InputLayer(
            (None,self.num_indicator_features),
            input_var=self.x_query_featurs,
        )
    
        rank_feats = [f[0] for f in enumerate(featuresNames) if f[1].startswith('Rank=')]
    
        self.denotation_join_feat_l = lasagne.layers.InputLayer(
            (None,self.num_indicator_features),
            input_var=self.x_denotaiton_features,#[:, rank_feats],
        )
        
        self.query_layer_l = lasagne.layers.DenseLayer(
            self.query_feat_l,
            name='query_lin',
            num_units=1,
            nonlinearity=lasagne.nonlinearities.linear,
        )
        
#         self.query_layer_l.b.get_value(borrow=True)[:] += 1
        
        self.query_output = lasagne.layers.get_output(
            lasagne.layers.reshape(self.query_layer_l, (-1,))
        )
        
        self.all_params = []
        
#         self.all_params += lasagne.layers.get_all_params(self.query_layer_l)
        
        self.aligned_queries = self.query_output[self.x_query_link_id]
        
        self.aligned_cosine = self.cosine_output[self.x_target_link_id]
        
        self.denotation_layer_l = lasagne.layers.DenseLayer(
            self.denotation_join_feat_l,
            name='denotation_lin',
            num_units=1,
            nonlinearity=lasagne.nonlinearities.linear,
            W=self.query_layer_l.W,
        )
        
#         self.denotation_layer_l.b.get_value(borrow=True)[:] += 1
        
        self.denotation_output = lasagne.layers.get_output(
            lasagne.layers.reshape(self.denotation_layer_l, (-1,)))
        
        self.all_params += lasagne.layers.get_all_params(self.denotation_layer_l)
        
        ###########################
        ## multiply the two parts of the join scores
        
        #  self.aligned_cosine
        self.unmerged_scores =  (
            ( #(self.aligned_queries) + 
            (self.denotation_output))# * 
#             (1 - self.x_denotaiton_features[:, self.impossible_query])
        )
        
        # this should be the fastest way to compute this
        # however it is creating matrices too large and isn't able to allocate enough memory or something
        
#         def mergingSelector(indx, outputs):
#             return T.set_subtensor(outputs[indx[0], T.arange(indx[1],indx[2])], 1)
        
#         merging_seq = T.concatenate([
#                 T.arange(self.x_denotation_ranges.shape[0]).reshape((self.x_denotation_ranges.shape[0], 1)),
#                 self.x_denotation_ranges,
#         ], axis=1)
        
#         self.merging_matrix, _ = theano.scan(
#             mergingSelector,
#             outputs_info=T.zeros((self.x_denotation_ranges.shape[0], self.denotation_output.shape[0])),
#             sequences=merging_seq,
#         )
        
#         self.merged_scores = T.dot(self.merging_matrix, self.unmerged_scores)


        def sloppyMathLogSum(vals):
            m = vals.max()
            return T.log(T.exp(vals - m).sum()) + m
    
        def mergingSum(indx, unmerged):
            return sloppyMathLogSum(unmerged[T.arange(indx[0], indx[1])]) #.mean()
    
        self.merged_scores, _ = theano.scan(
            mergingSum,
            sequences=[self.x_denotation_ranges],
            non_sequences=[self.unmerged_scores]
        )
        
#         self.merged_scores = self.cosine_output
        
        # prevents the softmax from blowing up
#         self.merged_rescaled = 10 * self.merged_scores / theano.gradient.disconnected_grad(abs(self.merged_scores).max())
        
        
        #############################
        ## Linear features combined
        #############################

        
#         self.linear_features_combined = lasagne.layers.concat(
#             [
                
#                comparedVLayers(self.target_out, self.source_aligned_l),
#                comparedVLayers(self.target_out, self.source_context_aligned_l),
#                comparedVLayers(self.target_out, self.source_surface_words_aligned_l),
                
#                comparedVLayers(self.target_title_out, self.source_aligned_l),
#                comparedVLayers(self.target_title_out, self.source_context_aligned_l),
#                comparedVLayers(self.target_title_out, self.source_surface_words_aligned_l),
             
                
# #                 comparedVLayers(self.target_embedding_out, self.source_aligned_l),
# #                 comparedVLayers(self.target_embedding_out, self.source_context_aligned_l),
# #                 comparedVLayers(self.target_embedding_out, self.source_surface_words_aligned_l),
                
#             lasagne.layers.reshape(self.target_matched_surface_input_l, ([0],1)),
#             self.target_matched_counts_input_l,
#              self.indicator_feat_l
#             ],
#             axis=1
#         )
        
#         self.linear_features_dens_l = lasagne.layers.DenseLayer(
#             self.linear_features_combined,
#             nonlinearity=lasagne.nonlinearities.linear,  # use tanh so that too large of values don't cause the softmax issues
#             num_units=1,
#             name='linear_final_l',
#             W=lasagne.init.Normal(mean=0.0),
#         )
        
#         self.linear_features_dens_l.W.get_value(borrow=True)[0:9] += 1.0  # set the word vecs positive
# #         lin_feat_W = self.linear_features_dens_l.W.get_value(borrow=True)
# #         lin_feat_add_one = np.eye(self.dim_compared_vec).reshape(self.dim_compared_vec**2, 1)
# #         lin_feat_W[self.dim_compared_vec*2                           :self.dim_compared_vec*2+  self.dim_compared_vec**2] += lin_feat_add_one
# #         lin_feat_W[self.dim_compared_vec*4+  self.dim_compared_vec**2:self.dim_compared_vec*4+2*self.dim_compared_vec**2] += lin_feat_add_one
# #         lin_feat_W[self.dim_compared_vec*6+2*self.dim_compared_vec**2:self.dim_compared_vec*6+3*self.dim_compared_vec**2] += lin_feat_add_one
        
#         self.linear_output = lasagne.layers.get_output(
#             lasagne.layers.reshape(self.linear_features_dens_l, ([0],))
#         )
        
#         # rescaled the output so we don't crash the softmax layer with a value that is too large
#         # just a hack
#         self.linear_output_rescaled = 10 * self.linear_output / theano.gradient.disconnected_grad(abs(self.linear_output).max())
        
        ########################################
        ## true output values
        ########################################
        
        self.unscaled_output = self.merged_scores
        
        
        # rescale the scores to prevent softmax from blowing up
        self.true_output = self.merged_scores #10 * self.unscaled_output / theano.gradient.disconnected_grad(abs(self.unscaled_output).max())#self.cosine_output # self.merged_rescaled #self.linear_output_rescaled
        
        
        
#         self.res_l = self.dotted_vectors / ((self.target_out.norm(1, axis=1) + .001) *
#                                             (self.source_aligned_l.norm(1, axis=1) + .001))


        #self.golds = self.res_cap[self.y_answer]

#         def maxOverRange(indx):
#             #return T.max(self.res_cap[T.arange(indx[0],indx[1])]) - self.res_cap[indx[2]]
#             #return -( self.res_l[indx[2]] - T.log(T.exp(self.res_l[T.arange(indx[0],indx[1])]).sum()) )
#             return -( self.res_l[indx[2]] - self.res_l[indx[0]])

#         # build a tensor to make a matrix with one set on each dimention
#         self.grouped, grouped_update = theano.scan(maxOverRange, sequences=self.y_grouping)

###########
#         def setSubSelector(indx, outputs):
#             return T.set_subtensor(outputs[T.arange(indx[0], indx[1]), indx[3]], 1)

#         num_target_samples = self.true_output.shape[0]

#         select_seq = T.concatenate([
#             self.y_grouping,
#             T.arange(self.y_grouping.shape[0]).reshape((self.y_grouping.shape[0], 1))
#         ], axis=1)

#         self.selecting_matrix, _ = theano.scan(
#             setSubSelector,
#             outputs_info=T.zeros((num_target_samples, self.y_grouping.shape[0])), #num_target_samples)),
#             #n_steps=self.y_grouping.shape[0]
#             sequences=select_seq,
#         )

#########
        
        
#         self.groupped_elems = T.dot(self.selecting_matrix[-1].T, 
#                                    T.exp(self.true_output))
#         self.groupped_res = T.log(self.groupped_elems)
            
        def lossSum(indx, res):
            return sloppyMathLogSum(res[T.arange(indx[0], indx[1])])
        
        self.groupped_res, _ = theano.scan(
            lossSum,
            sequences=[self.y_grouping],
            non_sequences=[self.true_output],
        )
        
        self.loss_vec = self.groupped_res - self.true_output[self.y_grouping[:,2]]
        
        if self.enable_boosting:
            self.loss_scalar = T.dot(self.y_boosted, self.loss_vec)
        else:
            self.loss_scalar = self.loss_vec.sum()

#         self.all_params = (
#             lasagne.layers.get_all_params(self.document_simple_sum_l) +
#             lasagne.layers.get_all_params(self.surface_context_pool1_l) +
#             lasagne.layers.get_all_params(self.surface_pool1_l) +
#             lasagne.layers.get_all_params(self.target_body_simple_sum_l) +
#             lasagne.layers.get_all_params(self.target_words_pool1_l) + 
#             lasagne.layers.get_all_params(self.linear_features_dens_l)
#         )
        
#         self.regularization = (self.linear_features_dens_l.W ** 2).sum() / 400

        # weight the positive samples more since there are fewer of them,
        # freaking hack
        #self.loss_vec = -(10 * self.y_score * T.log(self.res_cap) + (1.0 - self.y_score) * T.log(1.0 - self.res_cap))

        #self.loss_vec = T.nnet.binary_crossentropy(self.res_cap, self.y_score)

        #self.loss_vec = T.exp(T.max(self.res_cap - self.res_cap[self.y_answer] + .1, 0)) - 1  # TODO: maybe have some squared term here or something?

        # this one works reasonably well
        #self.loss_vec = - T.log((T.clip(self.res_cap[self.y_answer] - self.res_cap, -1.0, 0.4) + 1.0) / 1.5)

        #self.loss_vec = self.grouped

        #self.loss_vec = - T.log((T.clip(self.res_l[self.y_answer] - self.res_l, -40.0, 10.0) + 40.0) / 51.0)
        #self.loss_vec = T.max(self.res_l[self.y_answer] - self.res_l + .1, 0)

        self.updates = lasagne.updates.adadelta(
            self.loss_scalar / self.loss_vec.shape[0] , #+ self.regularization, 
            self.all_params)
        
        self.func_inputs = [
            self.x_document_input,
            self.x_surface_text_input, self.x_surface_context_input, self.x_document_id,
            self.x_target_input, self.x_matches_surface, self.x_matches_counts, self.x_link_id, 
            self.x_target_words, self.x_target_document_words, #self.x_indicator_features,
            self.x_denotaiton_features, self.x_query_featurs, self.x_query_link_id, self.x_denotation_ranges,
            self.x_target_link_id,
            self.y_answer, self.y_grouping, self.y_boosted
        ]
        
        self.func_outputs = [
            self.true_output,
            self.loss_vec.sum(),
            self.loss_scalar,
            self.loss_vec,
            #self.res_l,
        ]

        ################################################################3
        ## TODO: need to return the actual output layer instead of the res_cap, since that is something else now
        
        self.train_func = theano.function(
            self.func_inputs,
            self.func_outputs,
            updates=self.updates,
            on_unused_input='ignore',
        )

        self.test_func = theano.function(
            self.func_inputs,
            self.func_outputs,
            on_unused_input='ignore',
        )

    def reset_accums(self):
        self.current_documents = []
        self.current_surface_context = []
        self.current_surface_link = []
        self.current_link_id = []
        self.current_target_input = []
        self.current_target_words = []
        self.current_target_body_words = []
        self.current_target_matches_surface = []
        self.current_target_id = []
        self.current_target_goal = []
#         self.current_feat_indicators = []
        self.current_learning_groups = []
        self.learning_targets = []
        self.current_surface_target_counts = []
        self.current_boosted_groups = []
        
        self.current_queries = []
        self.current_denotations_feats_indicators = []
        self.current_denotations_related_query = []
        self.current_denotations_range = []
        self.current_denotation_targets_linked = []
        
        self.failed_match = []
            
    def compute_batch(self, isTraining=True, useTrainingFunc=True):
        if isTraining and useTrainingFunc:
            func = self.train_func
        else:
            func = self.test_func
        self.reset_accums()
        self.total_links = 0
        self.total_loss = 0.0
        self.total_boosted_loss = 0.0

        get_words = re.compile('[^a-zA-Z0-9 ]')
        get_link = re.compile('.*?\[(.*?)\].*?')

        for doc, queries in self.queries.iteritems():
            # skip the testing documents while training and vice versa
            if queries.values()[0]['training'] != isTraining:
                continue
            docid = len(self.current_documents)
            self.current_documents.append(self.wordvecs.tokenize(doc, length=self.document_length))
            for surtxt, targets in queries.iteritems():
                self.current_link_id.append(docid)
                surid = len(self.current_surface_link)
                self.current_surface_context.append(self.wordvecs.tokenize(get_words.sub(' ' , surtxt)))
                surlink = get_link.match(surtxt).group(1)
                self.current_surface_link.append(self.wordvecs.tokenize(surlink, length=self.sentence_length_short))
                surmatch = surlink.lower()
                surcounts = self.surface_counts.get(surmatch)
                if not surcounts:
                    self.failed_match.append(surmatch)
                    surcounts = {}
                target_body_words_input = []  # words from the target document
                target_words_input = []  # the words from the target title
                target_matches_surface = []
                target_inputs = []  # the target vector
                target_learings = []
                target_match_counts = []
                target_gold_loc = -1
                target_group_start = len(self.current_target_input)
#                 target_feat_indicators = []
                
                denotations_joint_indicators = []
                denotations_linked_query = []
                denotations_range = []
                
                denotation_target_linked = []
                
                queries_feats_indicators = []
                for ind in targets['query_vals']:
                    query_feats = np.zeros((self.num_indicator_features,), dtype='int8')
#                     query_feats[ind] = 1
                    queries_feats_indicators.append(query_feats)
                queries_len = len(targets['query_vals'])
                
                for target in set(targets['vals'].keys() +
                                 random.sample(self.documentvecs.reverse_word_location, self.num_negative_target_samples)
                                  ) - {None,}:
                    isGold = target in targets['gold']
                    wiki_title = WikiRegexes.convertToTitle(target)
                    cnt_wrds = self.page_content.get(wiki_title) #WikiRegexes.convertToTitle(target))
                    cnt = self.documentvecs.get_location(wiki_title)
                    if wiki_title == 'nil':
                        cnt = 0  # this is the stop symbol location
                    if cnt is None:
                        # were not able to find this wikipedia document
                        # so just ignore tihs result since trying to train on it will cause
                        # issues
                        if cnt_wrds is None:
                            # really know nothing
                            continue
                        else:
                            # we must not have had enough links to this document
                            # but still have the target text
                            cnt = 0
                    if isGold:
                        target_gold_loc = len(target_inputs)
                    target_body_words_input.append(cnt_wrds or [0]*self.sentence_length)
                    target_words_input.append(self.wordvecs.tokenize(get_words.sub(' ', target), length=self.sentence_length_short))
                    target_inputs.append(cnt)  
                    # page_content already tokenized
                    target_matches_surface.append(int(surmatch == target.lower()))
                    target_learings.append((targets, target))
                    target_match_counts.append(surcounts.get(wiki_title, 0))
                    
                    joint_indicators = []
                    query_idx = []
                    indicators_place = targets['vals'].get(target)
                    if indicators_place:
                        # [queries][indicator id]
                        for indx in xrange(len(indicators_place[1])):
                            local_feats = np.zeros((self.num_indicator_features,), dtype='int8')
                            local_feats[indicators_place[1][indx]] = 1
                            local_feats[targets['query_vals'][indx]] = 1  # features from the joint
                            if isGold:  #################################### hack
                                local_feats[-1] = 1
                            joint_indicators.append(local_feats)
#                             query_idx.append(len(self.current_queries) + indx)
                    else:
                        raise NotImplementedError()
                        for indx in xrange(queries_len):
                            local_feats = np.zeros((self.num_indicator_features,), dtype='int8')
                            local_feats[self.impossible_query] = 1
                            joint_indicators.append(local_feats)
                            query_idx.append(len(self.current_queries) + indx)
                            
                    start_range = len(denotations_joint_indicators) + len(self.current_denotations_feats_indicators)
                    denotations_joint_indicators += joint_indicators
                    denotations_linked_query += query_idx
                    denotations_range.append([start_range, start_range + len(joint_indicators)])
                    denotation_target_linked += [len(self.current_target_words) + len(target_words_input) - 1] * len(query_idx)
                    
                    
#                     indicators = np.zeros((self.num_indicator_features,), dtype='int8')
#                     if indicators_place:
                        
#                         indicators[indicators_place[1]] = 1
#                     target_feat_indicators.append(indicators)
                    
                    #if wiki_title not in surcounts:
                    #    print surcounts, wiki_title
                if target_gold_loc is not None or not isTraining:  # if we can't get the gold item
                    # contain the index of the gold item for these items, so it can be less then it
                    gold_loc = (len(self.current_target_goal) + target_gold_loc)
                    sorted_match_counts = [-4,-3,-2,-1] + sorted(set(target_match_counts))
                    #print sorted_match_counts
                    target_match_counts_indicators = [
                        [
                            int(s == sorted_match_counts[-1]),
                            int(s == sorted_match_counts[-2]),
                            int(s == sorted_match_counts[-3]),
                            int(0 < s <= sorted_match_counts[-4]),
                            int(s == 0),
                        ]
                        for s in target_match_counts
                    ]
                    self.current_target_goal += [gold_loc] * len(target_inputs)
                    self.current_target_input += target_inputs
                    self.current_target_id += [surid] * len(target_inputs)
                    self.current_target_words += target_words_input
                    self.current_target_matches_surface += target_matches_surface
                    self.current_surface_target_counts += target_match_counts_indicators
                    self.current_target_body_words += target_body_words_input
#                     self.current_feat_indicators += target_feat_indicators
                    
                    target_group_end = len(self.current_target_input)
                    self.current_learning_groups.append(
                        [target_group_start, target_group_end,
                         gold_loc])
                    self.current_boosted_groups.append(targets['boosted'])
                    
                    self.current_queries += queries_feats_indicators
                    
                    self.current_denotations_feats_indicators += denotations_joint_indicators
                    self.current_denotations_related_query += denotations_linked_query
                    self.current_denotations_range += denotations_range
                    
                    self.current_denotation_targets_linked += denotation_target_linked

                #self.current_target_goal.append(isGold)
                self.learning_targets += target_learings
            if len(self.current_target_id) > self.batch_size:
#                 return
                self.run_batch(func)
                if self.total_links > self.num_training_items:
                    return self.total_loss / self.total_links, self.total_boosted_loss / self.total_links

        if len(self.current_target_id) > 0:
            self.run_batch(func)

        return self.total_loss / self.total_links, self.total_boosted_loss / self.total_links

    def run_batch(self, func):
        res_vec, loss_sum, loss_boosted, loss_vec, = func(
            self.current_documents,
            self.current_surface_link, self.current_surface_context, self.current_link_id,
            self.current_target_input, self.current_target_matches_surface, self.current_surface_target_counts, self.current_target_id, 
            self.current_target_words, self.current_target_body_words, #self.current_feat_indicators,
            self.current_denotations_feats_indicators, self.current_queries, self.current_denotations_related_query, self.current_denotations_range,
            self.current_denotation_targets_linked,
            self.current_target_goal, self.current_learning_groups, self.current_boosted_groups,
        )
        self.check_params()
        self.total_links += len(self.current_target_id)
        self.total_loss += loss_sum
        self.total_boosted_loss += loss_boosted
        learned_groups = []  # right...dict not hashable....
        for i in xrange(len(res_vec)):
            # save the results from this pass
            l = self.learning_targets[i]
            if l[1] in l[0]['vals']:
                l[0]['vals'][ l[1] ][0] = float(res_vec[i]), 0#float(nn_outs[i])
            if l[0] not in learned_groups:
                learned_groups.append(l[0])
#         for group in learned_groups:
#             if group['gold']:
#                 correct = max(group['vals']) == group['vals'].get(group['gold'])
#                 group['boosted'] *= .4 if correct else 2.0
#                 if self.enable_cap_boosting:
#                     if group['boosted'] > 10:
#                         group['boosted'] = 10.0
#                     elif group['boosted'] < 0.1:
#                         group['boosted'] = 0.1
        self.reset_accums()

    def check_params(self):
        if any([np.isnan(v.get_value(borrow=True)).any() for v in self.all_params]):
            raise RuntimeError('nan in some of the parameters')



queries_exp = EntityVectorLinkExp()

In [ ]:
queries_exp.compute_batch()

In [77]:
len(queries_exp.current_denotations_range)

AttributeError: 'EntityVectorLinkExp' object has no attribute 'current_denotations_range'

In [51]:
len(queries_exp.current_denotations_related_query)

3133

In [84]:
queries_exp.query_layer_l.W.get_value(borrow=True).shape

(9954, 1)

In [55]:
loaded_weights = np.loadtxt('/home/matthew/data/external-wiki-weights-wplinks6.txt')

queries_exp.query_layer_l.W.get_value(borrow=True)[:] = 0
# queries_exp.query_layer_l.W.get_value(borrow=True)[0:loaded_weights.shape[0],0] = loaded_weights
queries_exp.query_layer_l.W.get_value(borrow=True)[-1] = 100

In [56]:
queries_exp.query_layer_l.W.get_value(borrow=True)

array([[   0.],
       [   0.],
       [   0.],
       ..., 
       [   0.],
       [   0.],
       [ 100.]], dtype=float32)

In [57]:
# run the evaluation on the testing data
queries_exp.num_training_items = 60000
queries_exp.compute_batch(False)
evalCurrentStateFahrni(False, queries_exp.num_training_items)
evalCurrentState(False, queries_exp.num_training_items)

KB: Prec = 443/476 = 0.930672268908, Rec = 443/485 = 0.913402061856, F1 = 0.921956295525
Pentagram (Pentagram album) [u'Pentagram_(Mezarkabul_album)', u'Pentagram (Mezarkabul album)']
Dead (Obituary album) [u'Knight of the Living Dead', u'Knight_of_the_Living_Dead']
-NIL- [u'Zmej']
The Fest [u'Monterrey Metal Fest', u'Monterrey_Metal_Fest']
The Fest [u'Monterrey Metal Fest', u'Monterrey_Metal_Fest']
True (TrinityRoots album) [u'Keep_It_True', u'Keep It True']
True (TrinityRoots album) [u'Keep_It_True', u'Keep It True']
True (TrinityRoots album) [u'Keep_It_True', u'Keep It True']
-NIL- [u'Cacophonous_Records', u'Cacophonous Records']
(485, 9, 0.9134020618556701)


(485, 9, 0.9134020618556701)

In [66]:
queries_exp.page_content.get(WikiRegexes.convertToTitle('Zmej'))

In [67]:
queries_exp.redirects.get(WikiRegexes.convertToTitle('Zmej'))

In [68]:
queries_exp.surface_counts.get(WikiRegexes.convertToTitle('Zmej'))

In [60]:
findWrongItems(False,20)

[{'gold': [u'Pentagram_(Mezarkabul_album)', u'Pentagram (Mezarkabul album)'],
  'ordered': [u'Pentagram (Pentagram album)',
   u'Pentagram (Indian band)',
   u'Pentagram (Gorgoroth album)',
   u'Relentless (Pentagram album)',
   u'Pentagram_(design_studio)',
   u'Pentagram (video game)',
   u'Pentagram (band)',
   u'Mezarkabul',
   u'Pentagram Chile',
   u'Pentagram',
   u'-NIL-',
   u'Pentagram (evil masquerade album)',
   u'Pentagram (Mezarkabul album)'],
  'text': u'Discography Albums * 1990 - [Pentagram] * 1992 - Trail Blazer * 1997 - Anatolia *'},
 {'gold': [u'Knight of the Living Dead', u'Knight_of_the_Living_Dead'],
  'ordered': [u'Dead (Obituary album)',
   u'Dead (Young Fathers album)',
   u'Per &quot;Dead&quot; Ohlin',
   u'Grateful Dead',
   u'Death',
   u'Living Dead (album)',
   u'Night of the Living Dead (film series)',
   u'Zombie',
   u'The Living Dead (album)',
   u'Escape on Venus',
   u'The Living Dead (TV series)',
   u'Living Dead',
   u'Knight Rider (2008 TV serie

In [86]:
queries_exp.query_layer_l.W.get_value(borrow=True).min()

-0.024544409

In [87]:
queries_exp.all_params

[query_lin.W, denotation_lin.b]

In [34]:
queries_exp.impossible_query

57

In [43]:
def evalCurrentState(trainingData=True, numSamples=50000):
    all_measured = 0
    all_correct = 0
    all_trained = 0
    all_wrong = 0
    for qu in queries.values():
        for en in qu.values():
            if en['training'] != trainingData:
                continue
            if en['gold']:
                if all_trained > numSamples:
                    break
                all_measured += 1
                all_trained += len(en['vals'].values())
                sv = sorted(en['vals'].items(), key=lambda x: x[1])
                #m = max(en['vals'].values())
                if sv[-1][0] in en['gold']:
                    all_correct += 1
                else:
                    if len(set(en['gold']) & set(en['vals'].keys())) != 0:
                        print sv[-1][0], en['gold']
                        all_wrong += 1
#                 for g in en['gold']:
#                     if en['vals'].get(en['gold'][0]) == m and m != 0:
#                         all_correct += 1
#                         break
           
    r = all_measured, all_wrong, float(all_correct) / all_measured
    print r
    return r

In [36]:
def evalCurrentStateRank(trainingData=True, numSamples=50000):
    all_measured = 0
    all_correct_place = 0
    p_counts = dict((k,0) for k in range(0,10))
    all_trained = 0
    for qu in queries.values():
        for en in qu.values():
            if en['training'] != trainingData:
                continue
            if en['gold']:
                if all_trained > numSamples:
                    break
                svals = sorted(en['vals'].values(), key=lambda x: 0 if not isinstance(x, tuple) else -x[0])
                gv = en['vals'][en['gold']]
                if gv == 0:
                    continue
                all_measured += 1
                for i in xrange(len(svals)):
                    if svals[i] == gv:
                        if i < 10:
                            p_counts[i] += 1
                        all_correct_place += i + 1
                        break

    r = all_measured, float(all_correct_place) / all_measured, p_counts
    print r
    return r


In [37]:
def evalCurrentStateF1(trainingData=True, numSamples=50000):
    correct = 0
    precDenom = 0
    recDenom = 0
    all_trained = 0
    all_measured = 0
    for qu in queries.values():
        if qu.values()[0]['training'] != trainingData:
            continue
        allGold = set()
        allChoosen = set()
        if all_trained > numSamples:
            break
        for en in qu.values():
            if en['gold']:  # we can eval this item
                all_measured += 1
                allGold.add(en['gold'])
                svals = sorted(en['vals'].values())
                picked = None
                for k,v in en['vals'].iteritems():
                    all_trained += 1
                    if v == svals[-1]:
                        picked = k
                allChoosen.add(picked)
                #if svals[0] == svals[1] and en['gold'] != picked:
                #    raise NotImplementedError()
#                 if en['gold'] == picked:
#                     correct += 1
#             if len(svals) > 5 and en['gold'] != picked:
#                 raise NotImplementedError()
        precDenom += len(allChoosen)
        recDenom += len(allGold)
        correct += len(allGold & allChoosen)
    correct = float(correct)
    prec = correct / precDenom
    rec = correct / recDenom
    f1 = 2 * prec * rec / (prec + rec)
    r = all_measured, 'Prec = {}/{} = {}, Rec = {}/{} = {}, F1 = {}'.format(
        correct, precDenom, prec, 
        correct, recDenom, rec, 
        f1)
    print r
    return r

In [38]:
from collections import defaultdict
def evalCurrentStateFahrni(trainingData=True, numSamples=50000):
    def renderF1(corr, precDenom, recDenom):
        prec = float(corr) / precDenom
        rec = float(corr) / recDenom
        return 'Prec = {}/{} = {}, Rec = {}/{} = {}, F1 = {}'.format(
            corr, precDenom, prec,
            corr, recDenom, rec,
            2 * prec * rec / (prec + rec)
        )
    
    counter = defaultdict(lambda: 0)
    for qu in queries.values():
        if qu.values()[0]['training'] != trainingData:
            continue
        for en in qu.values():
            if en['gold']:
                gold = en['gold']
                svals = sorted(en['vals'].items(), key=lambda x: x[1])
                picked = svals[-1][0]
                label = None
                if len(gold) == 1 and gold[0] == '-NIL-':
                    if picked == '-NIL-':
                        label = 'cNIL'
                    else:
                        label = 'wNIL_KB'
                elif picked in gold:
                    label = 'cKB'
                elif picked == '-NIL-':
                    label = 'wKB_NIL'
                else:
                    label = 'wKB_KB'
                counter[label] += 1
                
    print 'KB:', renderF1(counter['cKB'], counter['cKB'] + counter['wKB_KB'] + counter['wNIL_KB'], counter['cKB'] + counter['wKB_KB'] + counter['wKB_NIL'])
    if counter['cNIL']:
        print 'NIL:', renderF1(counter['cNIL'], counter['cNIL'] + counter['wKB_NIL'], counter['cNIL'] + counter['wNIL_KB'])

In [39]:
#sorted(queries.values()[0].values()[0]['vals'].values(), key=lambda x: 0 if not isinstance(x, tuple) else -x[0])

In [40]:
# import random
# def augmentTrainingData():
#     for quk in queries.keys():
#         qu = queries[quk]
#         for enk in qu.keys():
#             en = qu[enk]
#             if not en['gold']:
#                 del qu[enk]
#         if not qu:
#             del queries[quk]
#     for qu in queries.values():
#         training = random.random() > .15
#         for en in qu.values():
#             en['training'] = training
# augmentTrainingData()

In [59]:
def findWrongItems(trainingData=True, numSamples=50):
    ret = []
#     surface = set()
    for qu in queries.values():
        for ek, en in qu.items():
            if en['training'] != trainingData:
                continue
#             for e in en:
            if en['gold']:
                if len(ret) > numSamples:
                    return ret
                if True:#ek not in surface:
                    sv = sorted(en['vals'].items(), key=lambda x: x[1])
                    if sv[-1][0] not in en['gold'] and len(set(en['gold']) & set(en['vals'].keys())) != 0:
                        # got this wrong
                        ret.append({
                            'gold': en['gold'],
                            'ordered': [s[0] for s in sv][::-1],
                            'text': ek,
                        })
                        
#                     m = max(en['vals'].values())
#                     g = en['vals'].get(en['gold'][0], 0)
#                     if g != m and g != 0:
#                         ret[ek] = en
    return ret

In [79]:
findWrongItems(False)

[{'gold': [u'Single (music)', u'Single_(music)'],
  'ordered': [u'2009 Ho Chi Minh City',
   u'Single Session',
   u'Single National Park',
   u'Miriam Yeung discography',
   u'Single (band)',
   u'-NIL-',
   u'Georgism',
   u'Single-precision floating-point format',
   u'Single-cylinder engine',
   u'Scottish Aviation Pioneer',
   u'Single person',
   u'Single-player video game',
   u'CD single',
   u'Single_(music)',
   u'Single-camera setup',
   u'Single (baseball)',
   u'Single (Natasha Bedingfield song)',
   u'Single track (rail)',
   u'Single (New Kids on the Block and Ne-Yo song)'],
  'text': u') * By the Call of Seth ( 2006 , [Single] ) * Dystopia et Disturbia ( 2010 , Album )'},
 {'gold': [u'Aeolian mode', u'Aeolian_mode'],
  'ordered': [u'Aeolian (piano)',
   u'-NIL-',
   u'Aeolic verse',
   u'Aeolic Greek',
   u'Aeolian-Skinner',
   u'Aeolus',
   u'Vocalion Records',
   u'Aeolian harmony',
   u'Aeolian Quartet',
   u'Aeolis',
   u'Aeolian_mode',
   u'Aeolian processes',
   u

In [71]:
map

<function theano.scan_module.scan_views.map>

In [42]:
queries_exp.check_params()

In [44]:
queries_exp.num_training_items = 250000# 250000

In [35]:
queries_exp.num_training_items = 50000
%time print queries_exp.compute_batch()

UnboundLocalError: local variable 'target' referenced before assignment

In [74]:
from theano.compile.nanguardmode import NanGuardMode


def gg_test():
    self = queries_exp
    gg_f = theano.function(
            self.func_inputs,
            #self.func_outputs,
           T.grad(self.loss_vec.mean(), self.all_params) + 
        [lasagne.layers.get_output(self.cosine_combined), self.target_out, self.source_aligned_l, T.batched_dot(self.target_out + .001, self.source_aligned_l + .001), lasagne.layers.get_output(self.cosine_combined)],
            #updates=self.updates,
            on_unused_input='ignore',
            mode=NanGuardMode(nan_is_error=True, inf_is_error=True, big_is_error=False)
    )
    return gg_f(
            self.current_documents,
            self.current_surface_link, self.current_surface_context, self.current_link_id,
            self.current_target_input, self.current_target_matches_surface, self.current_surface_target_counts, self.current_target_id, 
            self.current_target_words, self.current_target_body_words, #self.current_feat_indicators,
            self.current_denotations_feats_indicators, self.current_queries, self.current_denotations_related_query, self.current_denotations_range,
            self.current_denotation_targets_linked,
            self.current_target_goal, self.current_learning_groups, self.current_boosted_groups,
        )

gg_res = gg_test()
gg_res

[array([[[[  2.25418884e-09,  -1.05965441e-08,  -1.08542056e-11,
             1.48679971e-10,  -3.70079661e-10,   8.10984879e-09,
             2.83995782e-09,  -7.58791163e-09,   1.44092704e-09,
            -5.21039156e-10,   3.83327015e-09,  -6.97350977e-09,
             7.41164152e-09,   2.56155741e-09,   7.06192482e-09,
            -2.65219424e-09,  -5.10543030e-09,  -2.67997929e-10,
            -6.05010930e-09,   1.29151201e-09,   3.90672739e-09,
            -1.60321534e-09,  -6.67960531e-09,  -1.06834408e-08,
            -1.52408985e-09,  -3.43769013e-09,   5.71510705e-09,
            -1.50977730e-09,  -1.31514755e-08,  -4.86911178e-09,
             1.11039511e-09,  -8.41305869e-09,   7.66608166e-10,
            -1.93872030e-09,  -2.17447549e-09,  -1.01494866e-08,
             2.11311568e-09,  -2.87067126e-09,  -1.14038743e-08,
             7.45462359e-09,  -2.55842503e-09,   5.26563948e-09,
            -9.17957443e-09,   2.05164019e-09,   7.46965778e-09,
            -1.56707554e-

In [49]:
len([f for f in featuresNames if f.startswith('Rank=')])

436

In [106]:
# run the evaluation on the testing data
queries_exp.num_training_items = 60000
queries_exp.compute_batch(False)
evalCurrentStateFahrni(False, queries_exp.num_training_items)

KB: Prec = 68/485 = 0.140206185567, Rec = 68/485 = 0.140206185567, F1 = 0.140206185567


In [60]:
exp_results = []

true_num_samples = 5000000 #queries_exp.num_training_items

for i in xrange(10):
    queries_exp.num_training_items = true_num_samples
    res = (i, queries_exp.compute_batch())
    print res
    exp_results.append(res)
    #if i % 2 == 1:
    exp_results.append(('training state', evalCurrentState(True, queries_exp.num_training_items)))
    queries_exp.num_training_items = 60000  # don't need that many samples to see how well it performs
    exp_results.append(('testing run', queries_exp.compute_batch(False)))
    exp_results.append(('testing state', evalCurrentState(False, queries_exp.num_training_items)))

(0, (0.085395781654700395, 0.085395781654700395))
(4480, 0.7267857142857143)
(485, 0.2536082474226804)
(1, (0.069128273271908375, 0.069128273271908375))
(4480, 0.7522321428571429)
(485, 0.2597938144329897)
(2, (0.056322899056998048, 0.056322899056998048))
(4480, 0.7863839285714286)
(485, 0.2597938144329897)


KeyboardInterrupt: 

In [47]:
evalCurrentStateF1(False,60000)

(485, 'Prec = 268.0/398 = 0.673366834171, Rec = 268.0/394 = 0.680203045685, F1 = 0.676767676768')


(485,
 'Prec = 268.0/398 = 0.673366834171, Rec = 268.0/394 = 0.680203045685, F1 = 0.676767676768')

In [50]:
evalCurrentStateFahrni(False,50000)

KB: Prec = 318/485 = 0.655670103093, Rec = 318/485 = 0.655670103093, F1 = 0.655670103093


In [103]:
queries_exp.denotation_layer_l.b.get_value(borrow=True)

array([ 4.02062559], dtype=float32)

In [104]:
queries_exp.query_layer_l.b.get_value(borrow=True)

array([ 2.63342738], dtype=float32)

In [107]:
queries_exp.query_layer_l.W.get_value(borrow=True).mean()

0.0024336253

In [88]:
exp_results

[(0, (0.085815984865510384, 0.085815984865510384)),
 ('training state', (4480, 0.7272321428571429)),
 ('testing run', (0.069930410587010397, 0.069930410587010397)),
 ('testing state', (485, 0.7381443298969073)),
 (1, (0.057293432118545068, 0.057293432118545068)),
 ('training state', (4480, 0.8100446428571428)),
 ('testing run', (0.064591320725843934, 0.064591320725843934)),
 ('testing state', (485, 0.7443298969072165)),
 (2, (0.047607379454377652, 0.047607379454377652)),
 ('training state', (4480, 0.8316964285714286)),
 ('testing run', (0.064104187438086269, 0.064104187438086269)),
 ('testing state', (485, 0.7443298969072165)),
 (3, (0.042031317627989527, 0.042031317627989527)),
 ('training state', (4480, 0.8428571428571429)),
 ('testing run', (0.059557031878569082, 0.059557031878569082)),
 ('testing state', (485, 0.7546391752577319)),
 (4, (0.036703072150937162, 0.036703072150937162)),
 ('training state', (4480, 0.8540178571428572)),
 ('testing run', (0.060888943361287196, 0.060888943

In [43]:
queries_exp.cosine_weighted.W.get_value(borrow=True)

array([[ 1.89486921]], dtype=float32)

In [44]:
queries_exp.embedding_W.get_value(borrow=True).shape

(146883, 300)

In [41]:
queries_exp.compute_batch()

KeyboardInterrupt: 

In [42]:
queries_exp.current_target

[[0, 1879, 3590, 6, 0, 0, 0, 0, 0, 0],
 [1879, 3592, 339, 0, 0, 0, 0, 0, 0, 0],
 [50, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1879, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1879, 3082, 0, 0, 0, 0, 0, 0, 0],
 [3591, 1879, 0, 0, 0, 0, 0, 0, 0, 0],
 [13978, 15792, 3482, 15793, 0, 0, 0, 0, 0, 0],
 [0, 1879, 49, 0, 0, 0, 0, 0, 0, 0],
 [1879, 125, 0, 0, 0, 0, 0, 0, 0, 0],
 [3595, 1305, 1879, 0, 0, 0, 0, 0, 0, 0],
 [2, 2402, 3, 1487, 1083, 0, 0, 0, 0, 0],
 [1879, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2441, 1549, 3568, 0, 0, 0, 0, 0, 0, 0],
 [17, 63, 3597, 3598, 0, 0, 0, 0, 0, 0],
 [1492, 2876, 196, 660, 1549, 3568, 136, 0, 0, 0],
 [50, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1549, 3568, 0, 0, 0, 0, 0, 0, 0, 0],
 [1635, 1549, 3568, 0, 0, 0, 0, 0, 0, 0],
 [1549, 3568, 49, 0, 0, 0, 0, 0, 0, 0],
 [13817, 15888, 15792, 5727, 302, 15793, 0, 0, 0, 0],
 [3568, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2323, 29, 0, 0, 0, 0, 0, 0, 0, 0],
 [50, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2323, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2323, 3167, 74, 160, 0, 0, 0, 0, 0, 0],
 

In [40]:
queries_exp.cosine_weighted.W.get_value(borrow=True)

array([[ 0.20918421],
       [ 0.32023922],
       [ 0.63108277],
       [ 0.3714537 ],
       [ 0.7353155 ],
       [ 0.85139668]], dtype=float32)

In [62]:
queries_exp.denotation_layer_l.b.get_value(borrow=True)

array([ 0.9999997], dtype=float32)

In [44]:
evalCurrentStateF1(False, 50000)

(2786, 'Prec = 1205.0/1511 = 0.797485109199, Rec = 1205.0/1491 = 0.808182427901, F1 = 0.802798134577')


(2786,
 'Prec = 1205.0/1511 = 0.797485109199, Rec = 1205.0/1491 = 0.808182427901, F1 = 0.802798134577')